Install dependencies required for this notebook:

In [1]:
%pip install pandas

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# import motion learning toolbox
import motion_learning_toolbox as mlt

Load data. The following methods expect positional columns to follow the pattern `<joint>_pos_<x/y/z>`, and orientational columns to follow the pattern `<joint>_rot_<x/y/z/w>` (order doesn't matter).

[`examples/data.csv`](examples/data.csv) contains raw data from a HTC Vive Pro setup; specifically, it is a sample from the [Who Is Alyx?](https://github.com/cschell/who-is-alyx) dataset.

In [3]:
time_unit = "ms" # if you have seconds, change to "s"
input_data = (pd.read_csv("examples/data.csv")
                # we cast the timestamp column into the timedelta type and set it as index; this is required for the resampling step.
                .assign(timestamp = lambda df: pd.to_timedelta(df["timestamp"], unit=time_unit)).set_index("timestamp"))
input_data.head()

,hmd_pos_x,hmd_pos_y,hmd_pos_z,hmd_rot_w,hmd_rot_x,hmd_rot_y,hmd_rot_z,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,...,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,right_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z
timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00,2.90,161.90,-17.27,0.65,-0.23,0.7,0.2,-16.90,141.36,-17.43,...,0.09,0.23,-0.72,-31.63,136.43,-10.80,0.4,0.32,0.83,-0.24
0 days 00:00:00.012000,2.92,161.91,-17.28,0.65,-0.23,0.7,0.2,-17.39,141.31,-17.03,...,0.09,0.24,-0.71,-31.60,136.44,-10.85,0.4,0.32,0.83,-0.24
0 days 00:00:00.024000,2.95,161.92,-17.30,0.65,-0.23,0.7,0.2,-17.85,141.23,-16.62,...,0.08,0.24,-0.71,-31.57,136.46,-10.91,0.4,0.32,0.83,-0.24
0 days 00:00:00.036000,2.96,161.93,-17.31,0.65,-0.22,0.7,0.2,-18.19,141.13,-16.25,...,0.08,0.24,-0.71,-31.53,136.49,-10.98,0.4,0.32,0.83,-0.24
0 days 00:00:00.049000,2.97,161.95,-17.33,0.65,-0.22,0.7,0.2,-18.47,141.04,-15.92,...,0.08,0.25,-0.71,-31.49,136.54,-11.05,0.4,0.32,0.82,-0.25


## Data Cleanup

### Resampling

It's a good idea to resample the tracking data to a constant framerate, as this ensures that input samples for machine learning models always represent the same time span. `resample` resamples position and rotation features in different ways:

- position features (that have `_pos_` in their column name) are interpolated linearily.
- rotation features (that have `_rot_` in their column name) are interpolated using Slerp.

In [4]:
resampled_data = mlt.resample(target_fps=5, data=input_data, joint_names=["hmd", "left_controller", "right_controller"])
resampled_data[:6]

,hmd_pos_x,hmd_pos_y,hmd_pos_z,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,hmd_rot_x,...,hmd_rot_z,hmd_rot_w,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,left_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z,right_controller_rot_w
timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00,2.900000,161.900000,-17.270000,-16.900000,141.360000,-17.430000,-31.630000,136.430000,-10.800000,-0.229382,...,0.199462,0.648252,0.089915,0.229782,-0.719317,0.649383,0.318585,0.826331,-0.238939,0.398232
0 days 00:00:00.200000,2.773636,162.138182,-17.634545,-17.608182,140.045455,-12.906364,-31.045455,137.581818,-12.326364,-0.220331,...,0.190286,0.650977,0.119848,0.317784,-0.660974,0.669151,0.299491,0.818610,-0.269542,0.409305
0 days 00:00:00.400000,2.007333,162.458667,-18.248667,-16.302667,128.166000,-10.926667,-31.300667,139.890667,-15.730000,-0.220071,...,0.180058,0.670215,0.094477,0.439165,-0.464460,0.763207,0.229803,0.784640,-0.359691,0.449614
0 days 00:00:00.600000,1.090000,163.020000,-18.590000,-11.430000,126.290000,-11.670000,-33.980000,141.480000,-22.220000,-0.210210,...,0.170170,0.690691,0.159864,0.479593,-0.409652,0.759355,-0.040030,0.680511,-0.560420,0.470353
0 days 00:00:00.800000,1.011111,163.070000,-18.877778,-12.276667,125.222222,-10.984444,-34.968889,142.710000,-24.751111,-0.209082,...,0.170159,0.700654,0.140049,0.500175,-0.390137,0.760266,-0.109974,0.629850,-0.629850,0.441002
0 days 00:00:01,0.927500,162.967500,-19.267500,-12.497500,126.175000,-11.152500,-34.745000,143.150000,-25.135000,-0.210147,...,0.170119,0.700491,0.149835,0.499451,-0.389572,0.759165,-0.107543,0.620262,-0.640270,0.440186


### Canonicalize Quaternions

A quaternion $q = a + b \mathbf{i} + c \mathbf{j} + d \mathbf{k}$ and its negation $-q = -a - b \mathbf{i} - c \mathbf{j} - d \mathbf{k}$ represent the same rotation in 3D space. This property is sometimes called the "double cover" feature of quaternions. When working with quaternions to represent rotations, it's common to normalize them to unit quaternions. In this context, $q$ and $-q$ will be antipodal points on the 4D unit sphere, both encoding the same rotation.

The double-cover property of quaternions can pose a challenge for machine learning models. For this reason, you can use `canonicalize_quaternions` to canonicalize quaternions. This method ensures that the scalar part (the "w" component) is always non-negative, so the same rotation will always yield the same values for x,y,z and w.

In [5]:
# for demonstration purposes, we just select the rotation columns of the hmd:
hmd_rot = input_data[["hmd_rot_x", "hmd_rot_y", "hmd_rot_z", "hmd_rot_w"]]

In [6]:
mlt.canonicalize_quaternions(hmd_rot, joint_names=["hmd"]).head()

,hmd_rot_x,hmd_rot_y,hmd_rot_z,hmd_rot_w
timestamp,,,,
0 days 00:00:00,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.012000,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.024000,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.036000,-0.219901,0.699685,0.199910,0.649708
0 days 00:00:00.049000,-0.219901,0.699685,0.199910,0.649708


This produces the same result even when we provide the negated rotation:

In [7]:
mlt.canonicalize_quaternions(-hmd_rot, joint_names=["hmd"]).head()

,hmd_rot_x,hmd_rot_y,hmd_rot_z,hmd_rot_w
timestamp,,,,
0 days 00:00:00,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.012000,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.024000,-0.229382,0.698118,0.199462,0.648252
0 days 00:00:00.036000,-0.219901,0.699685,0.199910,0.649708
0 days 00:00:00.049000,-0.219901,0.699685,0.199910,0.649708


## Data Encoding

The core features of this library target the encoding of tracking data. Identifying users based on their motions usually starts with a raw stream of positional and rotational data, which we term scene-relative (SR) data. While SR data is informative, it includes information that can distort the learning objectives of identification models. For instance, SR data includes not just user-specific characteristics but also information about the user's arbitrary position in the VR scene—features that don't contribute to user identity.

The following methods are based on our paper ["Comparison of Data Representations and Machine Learning Architectures for User Identification on Arbitrary Motion Sequences"](https://ieeexplore.ieee.org/document/10024474) (also available on [Arxiv](https://arxiv.org/abs/2210.00527)), you can find more details about the exact methods and reasoning there.

### Body Relative (BR) Encoding

The body-relative encoding transforms the coordinate system to a frame of reference attached to a specific joint of the user, typically their head/HMD, thereby filtering out some of the scene-specific noise.

> Position and orientation of the wrists are defined with respect to the head, which results in the wrists being positioned and orientated independently of the user's original scene position and orientation. The head's positional features and the rotation around the up axis become obsolete and are therefore removed, which only leaves one quaternion encoding the head's rotation around the horizontal axes. This has the effect that the data yields the same values for the same movement (e.g., waving), even if the user changes position or orientation within the scene in between.
> BR data consists of 18 features: (pos-x, pos-y, pos-z, rot-x, rot-y, rot-z, rot-w) $\times$  (wrist-left, wrist-right) + (rot-x, rot-y, rot-z, rot-w) $\times$ (head), all given with respect to the user's head as frame of reference. 

Make sure to specify the correct coordinate system, the one below should be correct for tracking data from Unity-like (left-hand) systems where the y-axis points up.


In [8]:
br_data = mlt.to_body_relative(resampled_data,
                                 target_joints=["left_controller", "right_controller"],
                                 reference_joint="hmd",
                                 coordinate_system={
                                    "forward": "x",
                                    "right": "z",
                                    "up": "y",
                                 })

br_data.head()

,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,left_controller_rot_w,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,right_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z,hmd_rot_w,hmd_rot_x,hmd_rot_y,hmd_rot_z
0,0.917561,-20.540000,19.779375,0.279674,-0.460463,0.629508,-0.559889,8.339203,-25.470000,34.126815,-0.326165,0.045569,0.857338,-0.395616,-0.061157,-0.012896,0.950715,0.303702
1,5.851482,-22.092727,20.088165,0.228996,-0.397806,0.704493,-0.541296,7.175955,-24.556364,33.472577,-0.313401,0.010005,0.859902,-0.402800,-0.061921,-0.013181,0.954679,0.290827
2,7.409258,-34.292667,18.274865,0.227103,-0.262559,0.850749,-0.394604,2.677522,-22.568000,33.295607,-0.238980,-0.092839,0.872182,-0.416615,-0.013120,-0.027618,0.958632,0.283000
3,6.381241,-36.730000,12.802991,0.216412,-0.168021,0.871662,-0.406375,-5.118304,-21.540000,34.883875,-0.131261,-0.416660,0.816760,-0.376920,0.034629,-0.034027,0.962109,0.268307
4,6.953341,-37.847778,13.802927,0.214708,-0.163732,0.884353,-0.380805,-8.353072,-20.360000,35.486372,-0.107198,-0.510073,0.761382,-0.385528,0.061673,-0.036805,0.961003,0.267048


Note that BR data doesn't include the reference_joint's position anymore, as it would always be (0,0,0).

### Body-Relative Velocity (BRV) Data

Computes the derivative of BR data over time, isolating the velocity component to focus machine learning models on actual user motion. `to_velocity` computes velocities for position and rotation features in different ways:

- the velocity for position features (that have `_pos_` in their column name) is just the difference between the current and the previous frame
- for rotation features (that have `_rot_` in their column name) it computes the delta rotation between the current and the previous frame, which isn't the same as subtracting the raw values.

In [9]:
brv_data = mlt.to_velocity(br_data)

brv_data.head()

,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,left_controller_rot_w,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,right_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z,hmd_rot_w,hmd_rot_x,hmd_rot_y,hmd_rot_z
0,0.917561,-20.540000,19.779375,0.279674,-0.460463,0.629508,-0.559889,8.339203,-25.470000,34.126815,-0.326165,0.045569,0.857338,-0.395616,-0.061157,-0.012896,0.950715,0.303702
1,5.851482,-22.092727,20.088165,0.228996,-0.397806,0.704493,-0.541296,7.175955,-24.556364,33.472577,-0.313401,0.010005,0.859902,-0.402800,-0.061921,-0.013181,0.954679,0.290827
2,7.409258,-34.292667,18.274865,0.227103,-0.262559,0.850749,-0.394604,2.677522,-22.568000,33.295607,-0.238980,-0.092839,0.872182,-0.416615,-0.013120,-0.027618,0.958632,0.283000
3,6.381241,-36.730000,12.802991,0.216412,-0.168021,0.871662,-0.406375,-5.118304,-21.540000,34.883875,-0.131261,-0.416660,0.816760,-0.376920,0.034629,-0.034027,0.962109,0.268307
4,6.953341,-37.847778,13.802927,0.214708,-0.163732,0.884353,-0.380805,-8.353072,-20.360000,35.486372,-0.107198,-0.510073,0.761382,-0.385528,0.061673,-0.036805,0.961003,0.267048


### Scene-Relative Velocity (SRV) Data

If you use the raw tracking data as input for `to_velocity`, you get Scene-Relative Velocity data. Note that SRV is usually not a desirable input for identification models, as it still encodes session-specific information.

In [10]:
srv_data = mlt.to_velocity(resampled_data)

srv_data.head()

,hmd_pos_x,hmd_pos_y,hmd_pos_z,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,hmd_rot_x,...,hmd_rot_z,hmd_rot_w,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,left_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z,right_controller_rot_w
timestamp,,,,,,,,,,,,,,,,,,,,,
0 days 00:00:00,2.900000,161.900000,-17.270000,-16.900000,141.360000,-17.430000,-31.630000,136.430000,-10.800000,-0.229382,...,0.199462,0.648252,0.089915,0.229782,-0.719317,0.649383,0.318585,0.826331,-0.238939,0.398232
0 days 00:00:00.200000,2.773636,162.138182,-17.634545,-17.608182,140.045455,-12.906364,-31.045455,137.581818,-12.326364,-0.220331,...,0.190286,0.650977,0.119848,0.317784,-0.660974,0.669151,0.299491,0.818610,-0.269542,0.409305
0 days 00:00:00.400000,2.007333,162.458667,-18.248667,-16.302667,128.166000,-10.926667,-31.300667,139.890667,-15.730000,-0.220071,...,0.180058,0.670215,0.094477,0.439165,-0.464460,0.763207,0.229803,0.784640,-0.359691,0.449614
0 days 00:00:00.600000,1.090000,163.020000,-18.590000,-11.430000,126.290000,-11.670000,-33.980000,141.480000,-22.220000,-0.210210,...,0.170170,0.690691,0.159864,0.479593,-0.409652,0.759355,-0.040030,0.680511,-0.560420,0.470353
0 days 00:00:00.800000,1.011111,163.070000,-18.877778,-12.276667,125.222222,-10.984444,-34.968889,142.710000,-24.751111,-0.209082,...,0.170159,0.700654,0.140049,0.500175,-0.390137,0.760266,-0.109974,0.629850,-0.629850,0.441002


### Body-Relative Acceleration (BRA) Data

Use `to_acceleration` to compute acceleration values. Behind the scenes this method actually just calls `to_velocity` twice, so it behaves like `to_velocity`.

In [11]:
bra_data = mlt.to_velocity(br_data)

bra_data.head()

,left_controller_pos_x,left_controller_pos_y,left_controller_pos_z,left_controller_rot_w,left_controller_rot_x,left_controller_rot_y,left_controller_rot_z,right_controller_pos_x,right_controller_pos_y,right_controller_pos_z,right_controller_rot_w,right_controller_rot_x,right_controller_rot_y,right_controller_rot_z,hmd_rot_w,hmd_rot_x,hmd_rot_y,hmd_rot_z
0,0.917561,-20.540000,19.779375,0.279674,-0.460463,0.629508,-0.559889,8.339203,-25.470000,34.126815,-0.326165,0.045569,0.857338,-0.395616,-0.061157,-0.012896,0.950715,0.303702
1,5.851482,-22.092727,20.088165,0.228996,-0.397806,0.704493,-0.541296,7.175955,-24.556364,33.472577,-0.313401,0.010005,0.859902,-0.402800,-0.061921,-0.013181,0.954679,0.290827
2,7.409258,-34.292667,18.274865,0.227103,-0.262559,0.850749,-0.394604,2.677522,-22.568000,33.295607,-0.238980,-0.092839,0.872182,-0.416615,-0.013120,-0.027618,0.958632,0.283000
3,6.381241,-36.730000,12.802991,0.216412,-0.168021,0.871662,-0.406375,-5.118304,-21.540000,34.883875,-0.131261,-0.416660,0.816760,-0.376920,0.034629,-0.034027,0.962109,0.268307
4,6.953341,-37.847778,13.802927,0.214708,-0.163732,0.884353,-0.380805,-8.353072,-20.360000,35.486372,-0.107198,-0.510073,0.761382,-0.385528,0.061673,-0.036805,0.961003,0.267048
